In [1]:
import duckdb
con = duckdb.connect(database='flight_weather_cancel.duckdb', read_only=False)

In [29]:
X = pd.read_parquet("/content/balanced_df_cleaned_cancellation.parquet")
X["FlightID"] = [i for i in range(len(X))]
y_cancel = pd.read_csv("y_cancel_stratified.csv")["Cancelled"]
X["cancelled"] = y_cancel

In [30]:
X.to_parquet("/content/balanced_df_cleaned_cancellation.parquet")

In [31]:
import os
import duckdb
import pandas as pd
from glob import glob

# --- Setup paths ---
con = duckdb.connect()
weather_folder = "/content/weather_files"
base_flight_file = "/content/balanced_df_cleaned_cancellation.parquet"

origin_dir = "/content/merged_batches_origin"
dest_dir = "/content/merged_batches_dest"
os.makedirs(origin_dir, exist_ok=True)
os.makedirs(dest_dir, exist_ok=True)

weather_files = sorted(glob(f"{weather_folder}/*.csv"))

# =======================================================
# PASS 1: ORIGIN WEATHER
# # =======================================================
for i, wf in enumerate(weather_files, 1):
    airport_code = os.path.splitext(os.path.basename(wf))[0].upper()
    print(f"\n[ORIGIN] Processing {airport_code} ({i}/{len(weather_files)})...")

    # Load only flights where this airport is the ORIGIN
    flights_subset = con.execute(f"""
        SELECT *
        FROM '{base_flight_file}'
        WHERE Origin = '{airport_code}'
    """).df()

    if flights_subset.empty:
        print(f"No origin flights for {airport_code}, skipping.")
        continue

    # Register tables
    con.register("flights", flights_subset)
    con.execute(f"""
        CREATE OR REPLACE TABLE weather AS
        SELECT *, '{airport_code}' AS airport_code FROM '{wf}';
    """)

    # Join with ±6h window on departure time
    merged_df = con.execute("""
        SELECT
            f.*,
AVG(CAST(w.WND AS DOUBLE))  AS o_WND,
AVG(CAST(w.CIG AS DOUBLE))  AS o_CIG,
AVG(CAST(w.VIS AS DOUBLE))  AS o_VIS,
AVG(CAST(w.TMP AS DOUBLE))  AS o_TMP,
AVG(CAST(w.DEW AS DOUBLE))  AS o_DEW,
AVG(CAST(w.SLP AS DOUBLE))  AS o_SLP,
AVG(CAST(w.AA1 AS DOUBLE))  AS o_AA1,
AVG(CAST(w.AA2 AS DOUBLE))  AS o_AA2,
MODE(w.AT1) AS o_AT1,
MODE(w.AT2) AS o_AT2,
MODE(w.AU1) AS o_AU1,
MODE(w.AU2) AS o_AU2,
MODE(w.AW1) AS o_AW1,
MODE(w.AW2) AS o_AW2,
MODE(w.GD1) AS o_GD1,
MODE(w.GD2) AS o_GD2,
AVG(CAST(w.OC1 AS DOUBLE))  AS o_OC1
        FROM flights f
        LEFT JOIN weather w
          ON w.airport_code = f.Origin
         AND w.DATE BETWEEN f.Dep_DateTime - INTERVAL 6 HOUR AND f.Dep_DateTime + INTERVAL 6 HOUR
        GROUP BY ALL;
    """).df()

    merged_df.to_parquet(f"{origin_dir}/{airport_code}_origin.parquet", index=False)
    print(f"✅ Saved origin batch: {airport_code}_origin.parquet")

# =======================================================
# PASS 2: DESTINATION WEATHER
# =======================================================
for i, wf in enumerate(weather_files, 1):
    airport_code = os.path.splitext(os.path.basename(wf))[0].upper()
    print(f"\n[DEST] Processing {airport_code} ({i}/{len(weather_files)})...")

    # Load only flights where this airport is the DESTINATION
    flights_subset = con.execute(f"""
        SELECT *
        FROM '{base_flight_file}'
        WHERE Dest = '{airport_code}'
    """).df()

    if flights_subset.empty:
        print(f"No destination flights for {airport_code}, skipping.")
        continue

    # Register tables
    con.register("flights", flights_subset)
    con.execute(f"""
        CREATE OR REPLACE TABLE weather AS
        SELECT *, '{airport_code}' AS airport_code FROM '{wf}';
    """)

    # Join with ±6h window on arrival time
    merged_df = con.execute("""
        SELECT
            f.*,
            AVG(CAST(wd.WND AS DOUBLE))  AS d_WND,
AVG(CAST(wd.CIG AS DOUBLE))  AS d_CIG,
AVG(CAST(wd.VIS AS DOUBLE))  AS d_VIS,
AVG(CAST(wd.TMP AS DOUBLE))  AS d_TMP,
AVG(CAST(wd.DEW AS DOUBLE))  AS d_DEW,
AVG(CAST(wd.SLP AS DOUBLE))  AS d_SLP,
AVG(CAST(wd.AA1 AS DOUBLE))  AS d_AA1,
AVG(CAST(wd.AA2 AS DOUBLE))  AS d_AA2,
MODE(wd.AT1) AS d_AT1,
MODE(wd.AT2) AS d_AT2,
MODE(wd.AU1) AS d_AU1,
MODE(wd.AU2) AS d_AU2,
MODE(wd.AW1) AS d_AW1,
MODE(wd.AW2) AS d_AW2,
MODE(wd.GD1) AS d_GD1,
MODE(wd.GD2) AS d_GD2,
AVG(CAST(wd.OC1 AS DOUBLE))  AS d_OC1
        FROM flights f
        LEFT JOIN weather wd
          ON wd.airport_code = f.Dest
         AND wd.DATE BETWEEN f.Arr_DateTime - INTERVAL 6 HOUR AND f.Arr_DateTime + INTERVAL 6 HOUR
        GROUP BY ALL;
    """).df()

    merged_df.to_parquet(f"{dest_dir}/{airport_code}_dest.parquet", index=False)
    print(f"✅ Saved destination batch: {airport_code}_dest.parquet")

print("\n🎉 All origin and destination weather merges complete!")



[ORIGIN] Processing ABE (1/362)...
✅ Saved origin batch: ABE_origin.parquet

[ORIGIN] Processing ABI (2/362)...
✅ Saved origin batch: ABI_origin.parquet

[ORIGIN] Processing ABQ (3/362)...
✅ Saved origin batch: ABQ_origin.parquet

[ORIGIN] Processing ABR (4/362)...
✅ Saved origin batch: ABR_origin.parquet

[ORIGIN] Processing ABY (5/362)...
✅ Saved origin batch: ABY_origin.parquet

[ORIGIN] Processing ACK (6/362)...
✅ Saved origin batch: ACK_origin.parquet

[ORIGIN] Processing ACT (7/362)...
✅ Saved origin batch: ACT_origin.parquet

[ORIGIN] Processing ACV (8/362)...
✅ Saved origin batch: ACV_origin.parquet

[ORIGIN] Processing ACY (9/362)...
✅ Saved origin batch: ACY_origin.parquet

[ORIGIN] Processing ADK (10/362)...
✅ Saved origin batch: ADK_origin.parquet

[ORIGIN] Processing ADQ (11/362)...
✅ Saved origin batch: ADQ_origin.parquet

[ORIGIN] Processing AEX (12/362)...
✅ Saved origin batch: AEX_origin.parquet

[ORIGIN] Processing AGS (13/362)...
✅ Saved origin batch: AGS_origin.par

In [32]:
import duckdb

# Connect to DuckDB
# con = duckdb.connect(database=':memory:')

# Read and merge all origin files into one big table
con.execute("""
    CREATE TABLE origin AS
    SELECT * FROM read_parquet('/content/merged_batches_origin/*.parquet');
""")

# Read and merge all destination files into another big table
con.execute("""
    CREATE TABLE dest AS
    SELECT * FROM read_parquet('/content/merged_batches_dest/*.parquet');
""")
print("read")

# Now join both on FlightID
merged_df = con.execute("""
    SELECT
        o.FlightID,
        o.*,
        d.d_WND, d.d_CIG, d.d_VIS, d.d_TMP, d.d_DEW, d.d_SLP,
        d.d_AA1, d.d_AA2, d.d_AT1, d.d_AT2, d.d_AU1, d.d_AU2,
        d.d_AW1, d.d_AW2, d.d_GD1, d.d_GD2, d.d_OC1
    FROM origin o
    LEFT JOIN dest d USING (FlightID);
""").df()

# Save the fully merged dataset
merged_df.to_parquet("/content/flights_cancelled_final_merged.parquet", index=False)

print("✅ Final dataset saved → flights_canclled_final_merged.parquet")


read
✅ Final dataset saved → flights_canclled_final_merged.parquet


In [33]:
merged_df.sort_values(by="FlightID")
merged_df.reset_index(drop=True, inplace=True)
merged_df.to_parquet("/content/flights_cancelled_final_merged.parquet", index=False)


In [36]:
merged_df = pd.read_parquet("/content/flights_cancelled_final_merged.parquet")

In [37]:
y = merged_df["cancelled"]
merged_df.drop(["cancelled"], axis=1, inplace=True)

In [22]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 6.3 MB/s eta 0:00:00


In [46]:
merged_df.drop(["FlightDate", "Dep_DateTime", "Arr_DateTime"], axis=1, inplace=True)

In [50]:
cat_cols = [
    "IATA_Code_Marketing_Airline",
    "Origin",
    "Dest",
    "Tail_Number",
    # Weather categorical columns
    "o_AT1", "o_AT2", "o_AU1", "o_AU2", "o_AW1", "o_AW2", "o_GD1", "o_GD2",
    "d_AT1", "d_AT2", "d_AU1", "d_AU2", "d_AW1", "d_AW2", "d_GD1", "d_GD2"
]

# -----------------------------
# 2. Convert all categorical columns to string dtype
# -----------------------------
for col in cat_cols:
    if col in merged_df.columns:
        merged_df[col] = merged_df[col].astype("string")

In [59]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

# Fit the encoder to the 'city' column and transform it
# The fit_transform() method learns the unique categories and then converts the column
merged_df['Origin_encoded'] = le.fit_transform(merged_df['Origin'])
merged_df['Dest_encoded'] = le.fit_transform(merged_df['Dest'])
merged_df['IATA_Code_Marketing_Airline_encoded'] = le.fit_transform(merged_df['IATA_Code_Marketing_Airline'])
merged_df['Tail_Number_encoded'] = le.fit_transform(merged_df['Tail_Number'])

merged_df.drop(['Origin', 'Dest', 'IATA_Code_Marketing_Airline', 'Tail_Number'], axis=1, inplace=True)

TypeError: Encoders require their input argument must be uniformly strings or numbers. Got ['NAType', 'str']

In [60]:
merged_df.drop(["Tail_Number"], axis=1, inplace=True)

In [64]:
import numpy as np
import pandas as pd

# Ensure all pd.NA become np.nan everywhere
merged_df = merged_df.replace({pd.NA: np.nan})

# Force all numeric columns to numeric dtype (if any are still object)
for col in merged_df.columns:
    if merged_df[col].dtype == 'object':
        try:
            merged_df[col] = pd.to_numeric(merged_df[col])
        except:
            pass  # Skip categorical ones

# Optional sanity check:
print(merged_df.isna().sum().sort_values(ascending=False).head(10))
print(merged_df.dtypes.value_counts())


d_AU2    121908
o_AU2    120297
d_AW2    118818
o_AW2    117324
o_OC1     91940
d_AU1     91602
d_AW1     89476
o_AU1     89107
d_OC1     89004
d_AA2     88345
dtype: int64
string[python]    19
float64           19
int64             10
Name: count, dtype: int64


In [68]:
import pandas as pd
import numpy as np

# Step 1 — Force all string columns back to object dtype
for col in merged_df.columns:
    if str(merged_df[col].dtype) == "string":
        merged_df[col] = merged_df[col].astype("object")

# Step 2 — Replace pd.NA and "<NA>" strings with np.nan everywhere
merged_df = merged_df.replace(["<NA>", pd.NA], np.nan)

# Step 3 — Optional: sanity check
print("Remaining pd.NA?", merged_df.isna().sum().sum())


Remaining pd.NA? 1522327


In [70]:
for col in merged_df.columns:
    if merged_df[col].isna().any():
        print(col, merged_df[col].dtype, merged_df[col].head(5).tolist())


o_WND float64 [5.552631578947367, 4.093749999999997, 2.3068965517241384, 4.905555555555555, 2.920000000000001]
o_CIG float64 [16566.684210526317, 431.546875, 111.82758620689656, 268.3333333333333, 8877.75]
o_VIS float64 [13139.894736842105, 10454.59375, 5928.517241379311, 15099.277777777777, 13070.0]
o_TMP float64 [27.400000000000002, 17.034374999999997, 12.21724137931035, 5.5500000000000025, 24.974999999999998]
o_DEW float64 [21.40526315789474, 14.778125000000005, 10.610344827586202, 3.0555555555555562, 22.705]
o_SLP float64 [1018.3763157894738, 1009.1703125000001, 1019.2327586206898, 1018.9509259259256, 1019.5899999999998]
o_AA1 float64 [1.8421052631578947, 0.575, 0.09137931034482759, 0.0, 0.0]
o_AA2 float64 [3.3000000000000003, 3.55, 0.3, 0.0, nan]
o_AT1 object ['3.0', '16.0', '16.0', '13.0', '13.0']
o_AT2 object ['16.0', '1.0', '13.0', '13.0', nan]
o_AU1 object ['0.0', '1.0', '0.0', '0.0', '0.0']
o_AU2 object ['3.0', '0.0', nan, nan, nan]
o_AW1 object ['61.0', '10.0', '10.0', '10.0

In [72]:
from sklearn.preprocessing import LabelEncoder

# Copy the dataframe to avoid accidental in-place damage
df = merged_df.copy()

# Identify categorical columns (string/object types)
cat_cols = df.select_dtypes(include=["object", "string"]).columns
print("Categorical columns to encode:", list(cat_cols))

# Initialize encoder
le = LabelEncoder()

# Encode each categorical column individually
for col in cat_cols:
    # Convert pd.NA / <NA> to a placeholder string
    df[col] = df[col].astype(str).replace({"<NA>": "missing", "nan": "missing"})
    df[col] = le.fit_transform(df[col])

print("✅ All categorical columns label encoded.")
print(df.dtypes)


Categorical columns to encode: ['Origin', 'Dest', 'IATA_Code_Marketing_Airline', 'o_AT1', 'o_AT2', 'o_AU1', 'o_AU2', 'o_AW1', 'o_AW2', 'o_GD1', 'o_GD2', 'd_AT1', 'd_AT2', 'd_AU1', 'd_AU2', 'd_AW1', 'd_AW2', 'd_GD1', 'd_GD2']
✅ All categorical columns label encoded.
FlightID                               float64
Origin                                   int64
Dest                                     int64
CRSDepTime                             float64
CRSArrTime                             float64
Distance                               float64
Month                                  float64
DayofMonth                             float64
DayOfWeek                              float64
IATA_Code_Marketing_Airline              int64
FlightID_1                             float64
o_WND                                  float64
o_CIG                                  float64
o_VIS                                  float64
o_TMP                                  float64
o_DEW                        

In [76]:
# --- Imports ---
import pandas as pd
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import numpy as np

# --- Define X and y ---
X = df.copy()

# --- Identify categorical columns (CatBoost needs names, not indices) ---
categorical_features = [
    "Origin", "Dest", "IATA_Code_Marketing_Airline",
    "o_AT1", "o_AT2", "o_AU1", "o_AU2", "o_AW1", "o_AW2", "o_GD1", "o_GD2",
    "d_AT1", "d_AT2", "d_AU1", "d_AU2", "d_AW1", "d_AW2", "d_GD1", "d_GD2"
]

# --- Train/Test split (stratified for class balance) ---
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

X[categorical_features] = X[categorical_features].astype("string").replace({pd.NA: np.nan})

print(f"Train size: {len(X_train):,}, Test size: {len(X_test):,}")
print(f"Class distribution in train:\n{y_train.value_counts(normalize=True)}")


Train size: 140,892, Test size: 35,223
Class distribution in train:
cancelled
True     0.500518
False    0.499482
Name: proportion, dtype: float64


In [77]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 140892 entries, 168490 to 53359
Data columns (total 48 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   FlightID                             140892 non-null  float64
 1   Origin                               140892 non-null  int64  
 2   Dest                                 140892 non-null  int64  
 3   CRSDepTime                           140892 non-null  float64
 4   CRSArrTime                           140892 non-null  float64
 5   Distance                             140892 non-null  float64
 6   Month                                140892 non-null  float64
 7   DayofMonth                           140892 non-null  float64
 8   DayOfWeek                            140892 non-null  float64
 9   IATA_Code_Marketing_Airline          140892 non-null  int64  
 10  FlightID_1                           140892 non-null  float64
 11  o_WND         

In [55]:
categorical_features = [
    "Origin", "Dest", "IATA_Code_Marketing_Airline", "Tail_Number",
    "o_AT1", "o_AT2", "o_AU1", "o_AU2", "o_AW1", "o_AW2", "o_GD1", "o_GD2",
    "d_AT1", "d_AT2", "d_AU1", "d_AU2", "d_AW1", "d_AW2", "d_GD1", "d_GD2"
]

In [78]:

# --- Create CatBoost Pool objects (handles NaNs + categorical internally) ---
train_pool = Pool(X_train, y_train, cat_features=categorical_features)
test_pool = Pool(X_test, y_test, cat_features=categorical_features)

# --- Define model ---
model = CatBoostClassifier(
    iterations=500,
    learning_rate=0.05,
    depth=8,
    loss_function='Logloss',
    eval_metric='F1',
    random_seed=42,
    early_stopping_rounds=50,
    verbose=100
)

# --- Fit model ---
model.fit(train_pool, eval_set=test_pool)

# --- Evaluate ---
y_pred = model.predict(X_test)
y_pred_prob = model.predict_proba(X_test)[:, 1]

print("\nClassification Report:")
print(classification_report(y_test, y_pred, digits=3))

print("\nROC-AUC Score:", round(roc_auc_score(y_test, y_pred_prob), 3))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# --- Feature Importance ---
feat_importance = pd.DataFrame({
    'Feature': X.columns,
    'Importance': model.get_feature_importance(train_pool)
}).sort_values(by='Importance', ascending=False)

print("\nTop 10 Important Features:")
print(feat_importance.head(10))


0:	learn: 0.7129597	test: 0.7087563	best: 0.7087563 (0)	total: 1.09s	remaining: 9m 6s
100:	learn: 0.7963485	test: 0.7934869	best: 0.7934869 (100)	total: 1m 33s	remaining: 6m 9s
200:	learn: 0.8149781	test: 0.8107966	best: 0.8107966 (200)	total: 3m 2s	remaining: 4m 31s
300:	learn: 0.8281692	test: 0.8219410	best: 0.8220475 (299)	total: 4m 36s	remaining: 3m 2s
400:	learn: 0.8385639	test: 0.8265810	best: 0.8265810 (400)	total: 6m 6s	remaining: 1m 30s
499:	learn: 0.8453064	test: 0.8305258	best: 0.8305357 (495)	total: 7m 35s	remaining: 0us

bestTest = 0.8305356882
bestIteration = 495

Shrink model to first 496 iterations.

Classification Report:
              precision    recall  f1-score   support

       False      0.817     0.864     0.840     17593
        True      0.856     0.807     0.831     17630

    accuracy                          0.835     35223
   macro avg      0.836     0.835     0.835     35223
weighted avg      0.836     0.835     0.835     35223


ROC-AUC Score: 0.914

Con

In [67]:
X_train

,FlightID,Origin,Dest,CRSDepTime,CRSArrTime,Distance,Month,DayofMonth,DayOfWeek,IATA_Code_Marketing_Airline,...,d_AU1,d_AU2,d_AW1,d_AW2,d_GD1,d_GD2,d_OC1,Origin_encoded,Dest_encoded,IATA_Code_Marketing_Airline_encoded
168490,51547,MCI,STL,1700,1800,237.0,1,8,1,WN,...,0.0,0.0,10.0,51.0,4.0,4.0,NaN,213,335,10
144437,155122,IAH,HOB,1215,1301,501.0,6,4,1,UA,...,<NA>,<NA>,<NA>,<NA>,0.0,2.0,14.300000,165,156,8
86352,49645,SAV,PIT,1930,2109,579.0,7,2,1,G4,...,0.0,1.0,61.0,90.0,1.0,2.0,13.900000,302,273,5
100205,41000,DFW,OMA,659,841,583.0,9,3,1,AA,...,1.0,1.0,61.0,91.0,2.0,3.0,NaN,93,251,0
95394,122474,SFO,PHX,2000,2255,651.0,1,22,1,WN,...,<NA>,<NA>,<NA>,<NA>,1.0,3.0,NaN,313,268,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17003,106867,MYR,LGA,943,1135,563.0,12,10,1,NK,...,<NA>,<NA>,<NA>,<NA>,1.0,1.0,7.200000,242,202,7
8573,130403,MSP,MDW,1925,2050,349.0,1,12,5,WN,...,1.0,0.0,71.0,71.0,4.0,4.0,14.338462,238,218,10
79268,68403,SAF,DEN,745,908,303.0,10,20,6,UA,...,<NA>,<NA>,<NA>,<NA>,0.0,<NA>,NaN,299,92,8
91910,41925,SEA,LAX,1145,1428,954.0,3,23,5,VX,...,0.0,0.0,10.0,51.0,2.0,3.0,9.300000,311,193,9
